In [1]:
import numpy as np
import torch
import matplotlib.pyplot as plt

In [3]:
class MyModel(torch.nn.Module):
    def __init__(self):
        super(MyModel, self).__init__()
        self.fc1 = torch.nn.Linear(1, 8)
        self.fc2 = torch.nn.Linear(8, 8)
        self.fc3 = torch.nn.Linear(8, 1)
        
        self.n = 3

    def forward(self, x):
        y1 = torch.nn.functional.relu(self.fc1(x))
        y2 = torch.nn.functional.relu(self.fc2(y1))
        y3 = self.fc3(y2)
        return y3
    
    def fc_interval(l, u, W, b):
        S = (W >= 0) * 1
        W1 = S * W    
        W2 = (1 - S) * W

        b = b.reshape(-1, 1)

        l_new = W1 @ l + W2 @ u + b
        u_new = W2 @ l + W1 @ u + b

        return l_new, u_new

    def relu_interval(l, u):
        l_new = torch.zeros(l.shape)
        u_new = u.clone()

        indices = (l>0)
        l_new[indices] = l[indices]

        indices = (u<0)
        u_new[indices] = 0

        return l_new, u_new
    
    def fc_d_interval(L, U, W):
        S = (W >= 0) * 1
        W1 = S * W    
        W2 = (1 - S) * W

        L_new = W1 @ L + W2 @ U
        U_new = W2 @ L + W1 @ U

        return L_new, U_new
    
    def relu_d_interval(l, u, L, U):
        L_relu = torch.zeros((l.shape))
        U_relu = torch.ones((l.shape))

        indices = (l>0)
        L_relu[indices] = 1

        indices = (u<0)
        U_relu[indices] = 0

        L_new = L * L_relu.view(-1,1)
        U_new = U * U_relu.view(-1,1)
        return L_new, U_new
    
    def forward_interval(low, high):
        l = torch.ones((1,1))*low   
        u = torch.ones((1,1))*high
        L = torch.eye(1)
        U = torch.eye(1)
        
        for i in range(self.n):
            W = list(model.named_parameters())[2*i][1].data
            b = list(model.named_parameters())[2*i+1][1].data
            l, u = fc_interval(l, u, W, b)
            L, U = fc_d_interval(L, U, W, b)
            if i != n-1:
                L, U = relu_d_interval(l, u, L, U)
                l, u = relu_interval(l, u)
        return l, u, L, U